<a href="https://colab.research.google.com/github/akapo/class-llm/blob/main/chapter2/langchain_step2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 실습 환경 설정

**필요한 패키지 설치**

In [ ]:
!pip install -qU openai
!pip install -qU langchain langchain-openai langchain-community langchainhub langchain-experimental
!pip install -qU huggingface_hub langchain-google-genai langchain-anthropic
!pip install -Uq sentence-transformers
!pip install -qU python-dotenv
!pip install -qU pymupdf pypdf
!pip install -qU numexpr
!pip install -qU faiss-cpu
!pip install -qU chromadb
!pip install -qU bs4
!pip install -qU google-search-results duckduckgo-search wikipedia
#!pip install -qU streamlit
#!pip install -qU streamlit-chat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/4

In [ ]:
#!pip install python-dotenv
import dotenv
dotenv.load_dotenv()

import os
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')

# 주의: API_KEY를 깃허브에 노출하면 LLM 업체는 해당키를 즉시 삭제함.
print("GOOGLE_API_KEY: " + GOOGLE_API_KEY[:20])
print("OPENAI_API_KEY: " + OPENAI_API_KEY[:20])
print("ANTHROPIC_API_KEY: " + ANTHROPIC_API_KEY[:20])

GOOGLE_API_KEY: AIzaSyCCtFS9bml5jdGI
OPENAI_API_KEY: sk-proj-FEDi6Ge6iQyR
ANTHROPIC_API_KEY: sk-ant-api03-YcGqyQQ


# **[LangChain]　3. Memory**

**ConversationBufferMemory 예제1**

In [ ]:
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(),
)

while True:
    user_message = input("You: ")

    if user_message == "끝":
        print("(대화 종료)")
        break

    ai_message = conversation.invoke(input=user_message)["response"]
    print(f"AI: {ai_message}")

You: 안녕? 나는 정진이야.
AI: 안녕, 정진! 만나서 반가워! 나는 AI야. 오늘 기분이 어때? 혹시 특별한 계획이 있니?
You: 아직 계획은 없어... 너 내 이름 기억해?
AI: 네, 정진! 너의 이름을 기억하고 있어. 앞으로도 계속 기억할게. 혹시 어떤 취미나 관심사가 있어? 함께 이야기해보면 좋을 것 같아!
You: 끝
(대화 종료)


**ConversationBufferMemory 예제2**

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import ConversationChain

# 1) 시스템 설정: 역할부여 정의
system_template = SystemMessagePromptTemplate.from_template("너는 지금 사람과 '1 to 50 게임'을 하는 AI야.")

# 2) ChatPromptTemplate 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    system_template,                                       # 역할부여
    MessagesPlaceholder(variable_name="chat_history"),     # 메모리 저장소 설정. ConversationBufferMemory의 memory_key 와 동일하게 설정
    HumanMessagePromptTemplate.from_template("{input}"),   # 사용자 메시지 injection
])

# 3) 메모리 정의
# `return_messages=True`를 사용해야 MessagesPlaceholder에 주입됨
# `chat_history`가 MessagesPlaceholder 이름과 일치해야함
memory = ConversationBufferMemory(memory_key="chat_history",
                                  ai_prefix="AI",
                                  human_prefix="Human",
                                  return_messages=True)

# 4) LLM 모델 정의
llm = ChatOpenAI(model_name='gpt-4o-mini')

# 5) LLMChain 정의
conversation = ConversationChain(
    llm=llm,       # LLM
    prompt=prompt, # Prompt
    memory=memory, # 메모리
    verbose=True,  # True 로 설정시 로그 출력
)

user_message = "내가 생각한 숫자가 무엇인지 맞춰봐"
while True:
    ai_message = conversation.invoke(input=user_message)["response"]
    print(f"AI: {ai_message}")

    user_message = input("You: ")
    if user_message == "끝":
        print("(대화 종료)")
        break



> Entering new ConversationChain chain...
Prompt after formatting:
System: 너는 지금 사람과 '1 to 50 게임'을 하는 AI야.
Human: 내가 생각한 숫자가 무엇인지 맞춰봐

> Finished chain.
AI: 좋아요! 1부터 50 사이의 숫자를 맞춰볼게요. 제 첫 번째 추측은 25입니다. 맞혔나요?
You: 보다커


> Entering new ConversationChain chain...
Prompt after formatting:
System: 너는 지금 사람과 '1 to 50 게임'을 하는 AI야.
Human: 내가 생각한 숫자가 무엇인지 맞춰봐
AI: 좋아요! 1부터 50 사이의 숫자를 맞춰볼게요. 제 첫 번째 추측은 25입니다. 맞혔나요?
Human: 보다커

> Finished chain.
AI: 알겠어요! 그럼 제 다음 추측은 38입니다. 이 숫자는 어떤가요?
You: 보다 작아


> Entering new ConversationChain chain...
Prompt after formatting:
System: 너는 지금 사람과 '1 to 50 게임'을 하는 AI야.
Human: 내가 생각한 숫자가 무엇인지 맞춰봐
AI: 좋아요! 1부터 50 사이의 숫자를 맞춰볼게요. 제 첫 번째 추측은 25입니다. 맞혔나요?
Human: 보다커
AI: 알겠어요! 그럼 제 다음 추측은 38입니다. 이 숫자는 어떤가요?
Human: 보다 작아

> Finished chain.
AI: 좋아요! 그럼 32는 어떤가요?
You: 정답이야


> Entering new ConversationChain chain...
Prompt after formatting:
System: 너는 지금 사람과 '1 to 50 게임'을 하는 AI야.
Human: 내가 생각한 숫자가 무엇인지 맞춰봐
AI: 좋아요! 1부터 50 사이의 숫자를 맞춰볼게요. 제 첫 번째 추측은 25입니다. 맞혔나요?
Human: 보다커

# **[LangChain]　4. Retrival**

## **1) Loader**

In [ ]:
# TextLoader & RecursiveCharacterTextSplitter
# Playground: https://chunkviz.up.railway.app/
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

length_function = len

splitter = CharacterTextSplitter(
    separator = "",  # Split character (default \n\n) <=
    chunk_size=400,
    chunk_overlap=40,
    length_function=length_function,
)
document = TextLoader("appendix-keywords.txt").load()
split_docs = splitter.split_documents(document)

# 총 분할된 도큐먼트 수
print(f'총 분할된 도큐먼트 수: {len(split_docs)}')
print(split_docs[0].page_content)
print('======')
print(split_docs[1].page_content)
print('======')
print(split_docs[2].page_content)

총 분할된 도큐먼트 수: 16
Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을
닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.
예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.
연관키워드: 토큰화, 자연어 처리, 구문 분석

Tokenizer

정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.
예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다.
연관키워드: 토큰화, 자연어 처리, 구문 분석

VectorStore

정의: 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템입니다. 이는 검색, 분류 및
터 형식으로 변환된 데이터를 저장하는 시스템입니다. 이는 검색, 분류 및 기타 데이터 분석 작업에 사용됩니다.
예시: 단어 임베딩 벡터들을 데이터베이스에 저장하여 빠르게 접근할 수 있습니다.
연관키워드: 임베딩, 데이터베이스, 벡터화

SQL

정의: SQL(Structured Query Language)은 데이터베이스에서 

In [ ]:
# TextLoader & RecursiveCharacterTextSplitter
# Playground: https://langchain-text-splitter.streamlit.app/
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=40,
    length_function=len,
)
document = TextLoader("appendix-keywords.txt").load()
split_docs = splitter.split_documents(document)

# 총 분할된 도큐먼트 수
print(f'총 분할된 도큐먼트 수: {len(split_docs)}')
print(split_docs[0].page_content)
print('======')
print(split_docs[1].page_content)
print('======')
print(split_docs[2].page_content)

총 분할된 도큐먼트 수: 19
Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token
Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.
예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.
연관키워드: 토큰화, 자연어 처리, 구문 분석

Tokenizer

정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.
예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다.
연관키워드: 토큰화, 자연어 처리, 구문 분석

VectorStore
VectorStore

정의: 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템입니다. 이는 검색, 분류 및 기타 데이터 분석 작업에 사용됩니다.
예시: 단어 임베딩 벡터들을 데이터베이스에 저장하여 빠르게 접근할 수 있습니다.
연관키워드: 임베딩, 데이터베이스, 벡터화

SQL

정의: SQL(Structured Query Language)은 데이터베이스에서 데이터를 관리하기 위한 프로그래밍 언어입니다. 데이터 조회, 수정, 삽입, 삭제 등 다양한 작업을 수행할 수 있습

In [ ]:
# PDFLoader
#!pip install -q pypdf
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("2024학년도 충청북도 중·고등학교 학업성적관리 도움자료.pdf")
docs = loader.load()

print(f'총 분할된 도큐먼트 수: {len(docs)}')
docs[15].page_content[:500]

총 분할된 도큐먼트 수: 78


'16\n2024학년도 충청북도 중·고등학교 학업성적관리 도움자료나. 전년도(학기) 문항 재출제, 출판사 제공 교사용 교재(CD) 및 참고서 전재 여부 등 확인\n다.  배점 표시, 역배점 지양 및 평가 문항수 조정과 배점 다양화로 석차 등급 과목의 경우 동점자 양산 방지 등 \n평가관리 공정성 확보\n라. 사전 문항 암시 주의\n마. 유사 선택교과 간 난이도 조정 권장 \n바.  출제범위와 수업 내용의 동일성 유지: 출제범위를 학급마다 다르게 알려주거나, 학급별로 가르친 내용이 \n다르지 않도록 주의\n사.  출제 원안지에 정답을 암시하는 별도 표시(글자 기울임, 다른 글자색, 글자체의 진함 등)가 없도록 철저히 \n사전 검토\n아.  서답형 문항 답안지 별도 제공: OMR카드에 서답형 문항 답안 작성 지양(학생의 답안지 교체 시 답안 작성 \n부담을 완화하기 위함)\n자. 수행평가의 일부로 실시하는 서·논술형 평가는 교과 수업 시간 중 실시가 원칙\n4. 평가 문항 출제 및 점검(문항 오류 방지)\n가. '

## **2) Embeddings**

In [ ]:
# OpenAI Embedding 모델
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

text="만나서 반가워요."
embeded = embeddings.embed_query(text)
print(str(len(embeded))+'차원')
print(embeded[:5])

1536차원
[-0.019351087510585785, -0.03914021700620651, 0.00637413514778018, -0.01435227133333683, -0.009520941413939]


In [ ]:
# 코사인 유사도 계산
import numpy as np
from numpy import dot
from numpy.linalg import norm

# 벡터 A와 벡터 B의 코사인 유사도 계산
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
# OpenAI Embeddings 모델 테스트
from langchain_openai import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings()

embeddings = embeddings_model.embed_documents(
    [
        '안녕하세요!',
        '어! 오랜만이에요',
        '이름이 어떻게 되세요?',
        '날씨가 추워요',
        'Hello LLM!'
    ]
)

embedded_query = embeddings_model.embed_query('첫인사를 하고 이름을 물어봤나요?')
print(str(len(embedded_query))+'차원')

for embedding in embeddings:
    print(cos_sim(embedding, embedded_query))

1536차원
0.8348159356519259
0.8153598883956
0.8843727343154585
0.7898927808309193
0.7469070089071632


In [ ]:
# HuggingFace의 jhgan/ko-sroberta-nli 모델 테스트
#!pip install -Uq sentence-transformers
from langchain.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

embeddings = embeddings_model.embed_documents(
    [
        '안녕하세요!',
        '어! 오랜만이에요',
        '이름이 어떻게 되세요?',
        '날씨가 추워요',
        'Hello LLM!'
    ]
)

embedded_query = embeddings_model.embed_query('첫인사를 하고 이름을 물어봤나요?')
print(str(len(embedded_query))+'차원')

for embedding in embeddings:
    print(cos_sim(embedding, embedded_query))

768차원
0.5899016189601531
0.4182631225980652
0.7240604521610333
0.05702662997392148
0.4316418328113528


In [ ]:
# GoogleGenerativeAIEmbeddings 모델 테스트
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings_model = GoogleGenerativeAIEmbeddings(model='models/embedding-001')

embeddings = embeddings_model.embed_documents(
    [
        '안녕하세요!',
        '어! 오랜만이에요',
        '이름이 어떻게 되세요?',
        '날씨가 추워요',
        'Hello LLM!'
    ]
)

embedded_query = embeddings_model.embed_query('첫인사를 하고 이름을 물어봤나요?')
print(str(len(embedded_query))+'차원')

for embedding in embeddings:
    print(cos_sim(embedding, embedded_query))

768차원
0.6051636939358793
0.620231830582371
0.7160855778483425
0.6574387494383126
0.55527388841408


In [ ]:
# Google GenAI의 Embedding 모델
# 텍스트를 특정 너비로 감싸는 데 사용되는 라이브러리를 가져옵니다.
import textwrap
# 과학 계산을 위한 라이브러리인 numpy를 가져옵니다.
import numpy as np
# 데이터 분석을 위한 라이브러리인 pandas를 가져옵니다.
import pandas as pd
# Google의 생성적 AI 모듈을 가져옵니다.
import google.generativeai as genai
# Google의 생성적 언어 모델을 가져옵니다.
import google.ai.generativelanguage as glm
# IPython 환경에서 Markdown 형식의 텍스트를 표시하기 위한 모듈을 가져옵니다.
from IPython.display import Markdown
# genai 라이브러리의 list_models 함수를 사용하여 사용 가능한 모델들의 목록을 가져옵니다.

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

for m in genai.list_models():
    # 각 모델의 supported_generation_methods 속성 중 'embedContent' 메소드를 지원하는지 확인합니다.
    if "embedContent" in m.supported_generation_methods:
        # 'embedContent' 메소드를 지원하는 모델의 이름을 출력합니다.
        print(m.name)

# 제목을 문자열로 선언합니다.
title = "The next generation of AI for developers and Google Workspace"
# 샘플 텍스트를 문자열로 선언합니다. 여기에는 제목과 본문이 포함되어 있습니다.
sample_text = (
    "Title: The next generation of AI for developers and Google Workspace"
    "\n"
    "Full article:\n"
    "\n"
    "Gemini API & Google AI Studio: An approachable way to explore and prototype with generative AI applications"
)

# 사용할 모델의 이름을 문자열로 선언합니다.
model = "models/embedding-001"
# genai.embed_content 함수를 사용하여 콘텐츠의 임베딩을 생성합니다.
# 이 함수는 모델, 콘텐츠, 작업 유형, 제목을 인자로 받습니다.
embeded = genai.embed_content(
    model=model, content=sample_text, task_type="retrieval_document", title=title
)

# 생성된 임베딩을 출력합니다.
print(str(len(embeded['embedding']))+'차원')
print(embeded)


text="진희는 강아지를 키우고 있습니다. 진희가 키우고 있는 동물은?"
# genai.embed_content 함수를 사용하여 콘텐츠의 임베딩을 생성합니다.
# 이 함수는 모델, 콘텐츠, 작업 유형, 제목을 인자로 받습니다.
embeded = genai.embed_content(
    model=model, content=text
)

# 생성된 임베딩을 출력합니다.
print(str(len(embeded['embedding']))+'차원')
print(embeded)

models/embedding-001
models/text-embedding-004
768차원
{'embedding': [0.03411343, -0.05517662, -0.020209055, -0.0041249567, 0.058917783, 0.014129515, 0.0045353593, 0.0014303668, 0.05976634, 0.08292115, 0.007162964, 0.0069041685, -0.053083427, -0.010905125, 0.0321402, -0.037163995, 0.050372455, 0.019348344, -0.037328612, 0.026647927, 0.030781753, -0.011288501, -0.031485256, -0.060248993, -0.026219442, -0.009794235, 0.006630139, -0.01846516, -0.026324715, 0.020442624, -0.06317684, 0.014559574, -0.052296035, 0.016451128, -9.720217e-05, -0.051706687, -0.0054406044, -0.056967627, 0.011144145, -0.009201792, -0.0021951047, -0.1099701, -0.011712193, 0.021221714, 0.009171804, -0.029621972, 0.034534883, 0.039578073, 0.019021519, -0.06269169, 0.039473332, 0.052403256, 0.061814185, -0.034507945, -0.009557816, -0.0049551064, 0.017839009, -0.021176832, 0.015043588, 0.015390569, -0.006334281, 0.043696404, -0.028341983, 0.028433999, 0.01472686, -0.06585564, -0.044533554, 0.0055523133, 0.035775978, 0.031

## **3) Vector Store**

**[벡터저장소]** FAISS(Facebook AI Similarity Search)

In [ ]:
# FAISS 유사도 기반 검색
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_openai import OpenAIEmbeddings

splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=40,
    length_function=len,
)

embeddings_model = OpenAIEmbeddings()

document = TextLoader("appendix-keywords.txt").load()
split_docs = splitter.split_documents(document)
print(f'총 분할된 도큐먼트 수: {len(split_docs)}')

#vector store
vs = FAISS.from_documents(
    split_docs,
    embedding = embeddings_model,
    #distance_strategy = DistanceStrategy.EUCLIDEAN_DISTANCE
    #distance_strategy = DistanceStrategy.COSINE
)
print(f"{vs.distance_strategy}\n")

query = "FAISS가 뭐야?"
docs = vs.similarity_search(query)
print(docs[0].page_content + "\n========\n")

query = "Pandas란?"
docs = vs.similarity_search(query)
print(docs[0].page_content)

총 분할된 도큐먼트 수: 19
EUCLIDEAN_DISTANCE

FAISS (Facebook AI Similarity Search)

정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.
예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.
연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화

Open Source

정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.
예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.
연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업

판다스 (Pandas)

정의: 판다스는 파이썬 프로그래밍 언어를 위한 데이터 분석 및 조작 도구를 제공하는 라이브러리입니다. 이는 데이터 분석 작업을 효율적으로 수행할 수 있게 합니다.
예시: 판다스를 사용하여 CSV 파일을 읽고, 데이터를 정제하며, 다양한 분석을 수행할 수 있습니다.
연관키워드: 데이터 분석, 파이썬, 데이터 처리

GPT (Generative Pretrained Transformer)


In [ ]:
# FAISS MMR(Maximum marginal relevances search) 검색
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_openai import OpenAIEmbeddings

splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=40,
    length_function=len,
)

embeddings_model = OpenAIEmbeddings()

document = TextLoader("appendix-keywords.txt").load()
split_docs = splitter.split_documents(document)
print(f'총 분할된 도큐먼트 수: {len(split_docs)}')

#vector store
vs = FAISS.from_documents(
    split_docs,
    embedding = embeddings_model,
    #distance_strategy = DistanceStrategy.EUCLIDEAN_DISTANCE
    distance_strategy = DistanceStrategy.COSINE
)
print(f"{vs.distance_strategy}\n")

query = "FAISS가 뭐야?"
mmr_docs = vs.max_marginal_relevance_search(query, k=2, fetch_k=5)
print(mmr_docs[0].page_content + "\n========\n")

query = "Pandas란?"
mmr_docs = vs.max_marginal_relevance_search(query, k=2, fetch_k=5)
print(mmr_docs[0].page_content)

총 분할된 도큐먼트 수: 19
COSINE

FAISS (Facebook AI Similarity Search)

정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.
예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.
연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화

Open Source

정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.
예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.
연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업

판다스 (Pandas)

정의: 판다스는 파이썬 프로그래밍 언어를 위한 데이터 분석 및 조작 도구를 제공하는 라이브러리입니다. 이는 데이터 분석 작업을 효율적으로 수행할 수 있게 합니다.
예시: 판다스를 사용하여 CSV 파일을 읽고, 데이터를 정제하며, 다양한 분석을 수행할 수 있습니다.
연관키워드: 데이터 분석, 파이썬, 데이터 처리

GPT (Generative Pretrained Transformer)


**[벡터저장소]** FAISS 로컬 DB Save & Load

In [ ]:
# 벡터저장소(FAISS)
#!pip install -q faiss-cpu
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=400,
    chunk_overlap=40,
    length_function=len,
)

embedding_model = OpenAIEmbeddings()
faiss_dir = "faiss-index"
doc_file_path = "appendix-keywords.txt"

if(os.path.isdir(faiss_dir) == False):
    document = TextLoader(doc_file_path).load()
    split_docs = splitter.split_documents(document)
    print(f'총 분할된 도큐먼트 수: {len(split_docs)}')

    vs = FAISS.from_documents(split_docs, embedding_model)
    vs.save_local(faiss_dir)
    print("saved")
else:
    vs = FAISS.load_local(faiss_dir, embedding_model, allow_dangerous_deserialization=True)
    print('loaded')

query = "FAISS가 뭐야?"
#query = "Pandas란?"
docs = vs.similarity_search(query)
print(docs[0].page_content)

loaded
FAISS (Facebook AI Similarity Search)

정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.
예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.
연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화

Open Source

정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.
예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.
연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업


**[벡터저장소]** Chroma DB

In [ ]:
# 벡터저장소(Chroma)
#!pip install -q chromadb
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

llm = ChatOpenAI(temperature=0, model_name='gpt-4o')
embedding_model = OpenAIEmbeddings()
db_dir = "chroma-db/"
doc_file_path = "소나기 - 황순원.pdf"

if(os.path.isdir(db_dir) == False):
    loader = PyPDFLoader(doc_file_path)
    docs = loader.load()
    vs = Chroma.from_documents(
        docs,
        embedding_model,
        persist_directory = db_dir,
        collection_name = 'shower',
        collection_metadata = {'hnsw:space': 'cosine'}, # 유사도 계산에 코사인 유사도를 사용
    )
    print("saved")
else:
    vs = Chroma(persist_directory=db_dir, embedding_function=embedding_model)
    print('loaded')

query = "소녀는 왜 이사를 오게 되었는가?"
docs = vs.similarity_search(query)
print(f'{len(docs)}개')
print(docs[0].page_content)

saved
4개
- 6 -소년은 소녀네가 이사해 오기 전에 벌써 어른들의 이야기를 들어서 , 윤 초시 손자(孫子)가 
서울서 사업에 실패해 가지고 고향에 돌아오지 않을 수 없게 되었다는 걸 알고 있었다 . 그
것이 이번에는 고향 집마저 남의 손에 넘기게 된 모양이었다 .
“왜 그런지 난 이사 가는 게 싫어졌다 . 어른들이 하는 일이니 어쩔 수 없지만 …….”
전에 없이, 소녀의 까만 눈에 쓸쓸한 빛이 떠돌았다 .
소녀와 헤어져 돌아오는 길에, 소년은 혼잣속으로 , 소녀가 이사를 간다는 말을 수없이 되
뇌어 보았다 . 무어 그리 안타까울 것도 서러울 것도 없었다 . 그렇건만 , 소년은 지금 자기가 
씹고 있는 대추알의 단맛을 모르고 있었다 .
이 날 밤, 소년은 몰래 덕쇠 할아버지네 호두밭으로 갔다.
낯에 봐 두었던 나무로 올라갔다 . 그리고 , 봐 두었던 가지를 향해 작대기를 내리쳤다 . 호두
송이 떨어지는 소리가 별나게 크게 들렸다 . 가슴이 선뜩했다 . 그러나 다음 순간, 굵은 호두
야 많이 떨어져라 , 많이 떨어져라 , 저도 모를 힘에 이끌려 마구 작대기를 내리 치는 것이
었다.
돌아오는 길에는 열 이틀 달이 지우는 그늘만 골라 디뎠다 . 그늘의 고마움을 처음 느꼈다 .
불룩한 주머니를 어루만졌다 . 호두송이를 맨손으로 깠다가는 옴이 오르기 쉽다는 말 같은 건 아
무렇지도 않았다 . 그저 근동에서 제일 가는 이 덕쇠 할아버지네 호두를 어서 소녀에게 맛보여야 
한다는 생각만이 앞섰다 .
그러다 , 아차 하는 생각이 들었다 . 소녀더러 병이 좀 낫거들랑 이사 가기 전에 한 번 개울
가로 나와 달라는 말을 못해 둔 것이었다 . 바보 같은것 , 바보 같은것 .
이튿날 , 소년이 학교에서 돌아오니 , 아버지가 나들이옷으로 갈아입고 닭 한 마리를 안고 
있었다 .
어디 가시느냐고 물었다 .
그 말에도 대꾸도 없이, 아버지는 안고 있는 닭의 무게를 겨냥해 보면서 ,
“이만하면 될까?”
어머니가 망태기를 내주며 ,
“벌써 며칠째 ‘걀걀’하고 알 날 자리를 보던데요 . 

## **4) Retrieval Chain**

**로그 활성화**

In [ ]:
import langchain
langchain.debug = False
langchain.verbose = False

**RetrievalQA**

In [ ]:
# RetrievalQA
#!pip install -q chromadb
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

llm = ChatOpenAI(temperature=0, model_name='gpt-4o-mini')
embeddings = OpenAIEmbeddings()
doc_file_path = "소나기 - 황순원.pdf"

loader = PyPDFLoader(doc_file_path)
docs = loader.load()
db = Chroma.from_documents(docs, embeddings)

retriever = db.as_retriever()

from langchain.chains import RetrievalQA
retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever)

query = "소녀는 왜 이사를 오게 되었는가?"
response = retriever.invoke({"query": query})
print(response)

{'query': '소녀는 왜 이사를 오게 되었는가?', 'result': '소녀는 윤 초시의 손자가 서울에서 사업에 실패하여 고향으로 돌아오게 되면서, 고향 집마저 남의 손에 넘기게 된 상황 때문에 이사를 오게 되었습니다.'}


In [ ]:
# 보너스: 검증된 프롬프트 가져오기
from langchain import hub
import pprint

prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

# 프롬프트 내용 출력
pprint.pp(prompt.messages[0].prompt.__dict__, indent=4)

{   'name': None,
    'input_variables': ['context', 'question'],
    'optional_variables': [],
    'input_types': {},
    'output_parser': None,
    'partial_variables': {},
    'metadata': None,
    'tags': None,
    'template': 'You are an assistant for question-answering tasks. Use the '
                'following pieces of retrieved context to answer the question. '
                "If you don't know the answer, just say that you don't know. "
                'Use three sentences maximum and keep the answer concise.\n'
                'Question: {question} \n'
                'Context: {context} \n'
                'Answer:',
    'template_format': 'f-string',
    'validate_template': False}


In [ ]:
# LangSmith 를 이용한 모니터링
os.environ["LANGCHAIN_PROJECT"] = "chain-monitor"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="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"
os.environ["LANGCHAIN_TRACING_V2"]="true"

In [ ]:
# RetrievalQA (chain_type="stuff")
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

llm = ChatOpenAI(temperature=0, model_name='gpt-4o-mini')
embeddings = OpenAIEmbeddings()
doc_file_path = "소나기 - 황순원.pdf"

loader = PyPDFLoader(doc_file_path)
docs = loader.load()
db = Chroma.from_documents(docs, embeddings)

retriever = db.as_retriever()

from langchain.chains import RetrievalQA
retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever)

query = "소설 끝에서 소녀는 어떠한 결말을 맞이한거야?"
response = retriever.invoke({"query": query})
print(response)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "소설 끝에서 소녀는 어떠한 결말을 맞이한거야?"
}
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "- 3 -돌아다보니 , 소녀는 지금 자기가 지나쳐 온 허수아비를 흔들고 있다. 좀 전 허수아비보다 \n더 우쭐거린다 .\n논이 끝난 곳에 도랑이 하나 있었다 . 소녀가 먼저 뛰어 건넜다 .\n거기서부터 산 밑까지는 밭이었다 .\n수숫단을 세워 놓은 밭머리를 지났다 .\n“저게 뭐니?”\n“원두막 .”\n“여기 참외, 맛있니 ?”\n“그럼, 참외 맛도 좋지만 수박 맛은 더 좋다.”\n“하나 먹어 봤으면 .”\n소년이 참외 그루에 심은 무우밭으로 들어가 , 무우 두 밑을 뽑아 왔다. 아직 밑이 덜 들어 \n있었다 . 잎을 비틀어 팽개친 후, 소녀에게 한개 건넨다 . 그리고는 이렇게 먹어야 한다는 듯\n이, 먼저 대강이를 한 입 베물어 낸 다음, 손톱으로 한 돌이 껍질을 벗겨 우쩍 깨문다 .\n소녀도 따라 했다. 그러나 , 세 입도 못 먹고,\n“아, 맵고 지려.”\n하며 집어던지고 만다.\n“참, 맛 없어 못 먹겠다 .”\n소년이 더 멀리 팽개쳐 버렸다 .\n산이 가까워졌다 .\n단풍이 눈에 따가웠다 .\n“야아!”\n소녀가 산을 향해 달려갔다 . 이번은 소년이 뒤따라 달리지 않았다 . 그러고도 곧 소녀보다 \n더 많은 꽃을 꺾었다 .\n“이게 들국화 , 이게 싸리꽃 , 이게 도라지꽃 ,…….”\n“도라지꽃이 

In [ ]:
# LangSmith 를 이용한 모니터링
# RetrievalQA (chain_type="map_reduce")
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

llm = ChatOpenAI(temperature=0, model_name='gpt-4o')
embeddings = OpenAIEmbeddings()
doc_file_path = "소나기 - 황순원.pdf"

loader = PyPDFLoader(doc_file_path)
docs = loader.load()
db = Chroma.from_documents(docs, embeddings)

retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 3})

from langchain.chains import RetrievalQA
retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=retriever)

query = "소설 끝에서 소녀는 어떠한 결말을 맞이한거야?"
response = retriever.invoke({"query": query})
print(response)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "소설 끝에서 소녀는 어떠한 결말을 맞이한거야?"
}
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "- 3 -돌아다보니 , 소녀는 지금 자기가 지나쳐 온 허수아비를 흔들고 있다. 좀 전 허수아비보다 \n더 우쭐거린다 .\n논이 끝난 곳에 도랑이 하나 있었다 . 소녀가 먼저 뛰어 건넜다 .\n거기서부터 산 밑까지는 밭이었다 .\n수숫단을 세워 놓은 밭머리를 지났다 .\n“저게 뭐니?”\n“원두막 .”\n“여기 참외, 맛있니 ?”\n“그럼, 참외 맛도 좋지만 수박 맛은 더 좋다.”\n“하나 먹어 봤으면 .”\n소년이 참외 그루에 심은 무우밭으로 들어가 , 무우 두 밑을 뽑아 왔다. 아직 밑이 덜 들어 \n있었다 . 잎을 비틀어 팽개친 후, 소녀에게 한개 건넨다 . 그리고는 이렇게 먹어야 한다는 듯\n이, 먼저 대강이를 한 입 베물어 낸 다음, 손톱으로 한 돌이 껍질을 벗겨 우쩍 깨문다 .\n소녀도 따라 했다. 그러나 , 세 입도 못 먹고,\n“아, 맵고 지려.”\n하며 집어던지고 만다.\n“참, 맛 없어 못 먹겠다 .”\n소년이 더 멀리 팽개쳐 버렸다 .\n산이 가까워졌다 .\n단풍이 눈에 따가웠다 .\n“야아!”\n소녀가 산을 향해 달려갔다 . 이번은 소년이 뒤따라 달리지 않았다 . 그러고도 곧 소녀보다 \n더 많은 꽃을 꺾었다 .\n“이게 들국화 , 이게 싸리꽃 , 이게 도라지꽃 ,…….”\n“도라지꽃이 

In [ ]:
# LCEL(LangChain Expression Language) 버전1
# FAISS 벡터스토어 COSINE_DISTANCE 사용
# rlm/rag-prompt 프롬프트 사용
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain import hub

embedding_model = OpenAIEmbeddings()
loader = PyPDFLoader("소나기 - 황순원.pdf")
docs = loader.load()

vectorstore = FAISS.from_documents(
    docs,
    embedding = embedding_model,
    distance_strategy = DistanceStrategy.COSINE
)
print(f"{vectorstore.distance_strategy}\n")

retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")  # langchainhub에서 검증된 prompt를 가져옴.

model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

#print(chain.invoke("소녀는 왜 이사를 오게 되었는가?"))
#print(chain.invoke("소녀는 소년을 어떻게 생각하고 있는가?"))
#print(chain.invoke(input="소설 끝에서 소녀는 어떠한 결말을 맞이한거야?", config={'callbacks': [ConsoleCallbackHandler()]}))
print(chain.invoke(input="소설 끝에서 소녀는 어떠한 결말을 맞이한거야? 제시된 context에서 근거를 찾아 설명해줘"))

소설 "소나기"에서 소녀는 이사 가기 전에 아픈 상태로 남겨지며, 결국 소년과의 이별을 맞이합니다. 소년은 소녀가 이사 간다는 사실에 슬픔을 느끼고, 소녀의 아픔과 이사에 대한 안타까움을 깊이 생각하게 됩니다. 소녀의 결말은 이사와 함께 소년과의 소중한 추억을 남기고, 그들의 관계가 끝나는 것으로 해석됩니다.


In [ ]:
# LCEL(LangChain Expression Language) 버전2
# FAISS 벡터스토어 EUCLIDEAN_DISTANCE 사용
# Prompt를 직접 설계하는 방법
from operator import itemgetter

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.callbacks.tracers import ConsoleCallbackHandler

loader = PyPDFLoader("소나기 - 황순원.pdf")
docs = loader.load()
vectorstore = FAISS.from_documents(docs, embedding=OpenAIEmbeddings())
print(f"{vectorstore.distance_strategy}\n")

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model_name='gpt-4o', temperature=0)

# LCEL에서 chain_type='map_reduce'로 지정하는 방법 아직 못찾아 냄.
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

#print(chain.invoke("이 소설에서 주인공의 이름은?"))
#print(chain.invoke("소녀가 대추를 딴 이유는?"))

# 'gpt-4o-mini' 와 'gpt-4o' 의 추론 능력의 차이를 확인할 수 있는 질문
print(chain.invoke("소설 끝에서 소녀가 어떠한 결말을 맞이한거야? 제시된 context에서 근거를 찾아 설명해줘"))
#chain.invoke(input="소설 끝에서 소녀가 어떠한 결말을 맞이한거야? 제시된 context에서 근거를 찾아 설명해줘", config={'callbacks': [ConsoleCallbackHandler()]})

EUCLIDEAN_DISTANCE

소설의 끝에서 소녀는 죽음을 맞이합니다. 제시된 문맥에서 이를 확인할 수 있는 근거는 다음과 같습니다:

1. **아버지의 대화**:
   - "윤 초시 댁도 말이 아니야, 그 많던 전답을 다 팔아 버리고, 대대로 살아오던 집마저 남의 손에 넘기더니, 또 악상까지 당하는걸 보면……."
   - 여기서 "악상"은 자식이 부모보다 먼저 죽는 것을 의미합니다. 따라서 윤 초시 댁에서 자식이 죽었다는 것을 알 수 있습니다.

2. **어머니의 대화**:
   - "증손(曾孫)이라곤 계집애 그 애 하나뿐이었지요?"
   - "그렇지, 사내 애 둘 있던 건 어려서 잃어버리고 ……."
   - "어쩌면 그렇게 자식복이 없을까."
   - 이 대화에서 소녀가 윤 초시의 증손녀임을 확인할 수 있습니다.

3. **소녀의 마지막 말**:
   - "글쎄, 죽기 전에 이런 말을 했다지 않아? 자기가 죽거든 자기 입던 옷을 꼭 그대로 입혀서 묻어 달라고 ……."
   - 소녀가 죽기 전에 자신의 옷을 그대로 입혀서 묻어 달라고 말했다는 내용은 소녀가 죽음을 맞이했음을 명확히 보여줍니다.

따라서, 소설의 끝에서 소녀는 병으로 인해 죽음을 맞이한 것으로 결말이 맺어집니다.


# **[LangChain]　5. Agent**

Agent의 작동원리 ReAct 개념 예제

In [ ]:
import langchain
langchain.verbose = True

import pprint
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
tools = load_tools(["terminal"], allow_dangerous_tools=True)

prompt = hub.pull("hwchase17/react")
pprint.pp(prompt.template, indent=4, width=100)  # <=

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({"input": "sample_data 디렉터리에 있는 파일 목록을 알려줘"})
print(result["output"])

('Answer the following questions as best you can. You have access to the following tools:\n'
 '\n'
 '{tools}\n'
 '\n'
 'Use the following format:\n'
 '\n'
 'Question: the input question you must answer\n'
 'Thought: you should always think about what to do\n'
 'Action: the action to take, should be one of [{tool_names}]\n'
 'Action Input: the input to the action\n'
 'Observation: the result of the action\n'
 '... (this Thought/Action/Action Input/Observation can repeat N times)\n'
 'Thought: I now know the final answer\n'
 'Final Answer: the final answer to the original input question\n'
 '\n'
 'Begin!\n'
 '\n'
 'Question: {input}\n'
 'Thought:{agent_scratchpad}')


> Entering new AgentExecutor chain...
I need to check the contents of the "sample_data" directory to list the files it contains. I'll use the terminal to do this.

Action: terminal  
Action Input: ls sample_data  Executing command:
 ls sample_data
anscombe.json
california_housing_test.csv
california_housing_train.csv
mnist_

/usr/local/lib/python3.10/dist-packages/langchain_community/tools/shell/tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


I now know the final answer, which is the list of files in the "sample_data" directory.  
Final Answer: The files in the "sample_data" directory are: anscombe.json, california_housing_test.csv, california_housing_train.csv, mnist_test.csv, mnist_train_small.csv, README.md.

> Finished chain.
The files in the "sample_data" directory are: anscombe.json, california_housing_test.csv, california_housing_train.csv, mnist_test.csv, mnist_train_small.csv, README.md.


**사용자 정의 Tool 예시 #2**

In [ ]:
from langchain.tools import Tool
from langchain.agents import AgentExecutor, create_react_agent, load_tools

def Komifoma_func(param):
        return "박정진"

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

tools = [
    Tool.from_function(
        func=Komifoma_func,
        name="Komifoma_Answer",
        description="Komifoma World에 대한 모든 질문에 대한 답"
    ),
]

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke({"input": "Komifoma를 누가 만들었나요?"})
result = agent_executor.invoke({"input": "Komifoma를 누가 끝장냈나요?"})



> Entering new AgentExecutor chain...
I need to find out who created Komifoma. 
Action: Komifoma_Answer
Action Input: "Komifoma를 누가 만들었나요?"박정진I now know the final answer.  
Final Answer: Komifoma는 박정진에 의해 만들어졌습니다.

> Finished chain.


> Entering new AgentExecutor chain...
I need to find out who ended Komifoma. 
Action: Komifoma_Answer
Action Input: "Komifoma를 누가 끝장냈나요?"박정진I now know the final answer.  
Final Answer: 박정진

> Finished chain.


**사용자 정의 Tool 예시 #2**

In [ ]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.agents import AgentExecutor, create_openai_functions_agent

class WeddingInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

def concat(a: int, b: int) -> int:
    """두 수의 결혼"""
    return f"{a}{b}"


calculator = StructuredTool.from_function(
    func=concat,
    name="ConCat",
    description="두 수의 결혼 결과",
    args_schema=WeddingInput,
    return_direct=True,
)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
prompt = hub.pull("hwchase17/openai-functions-agent")
tools = [calculator]

print(calculator.name)
print(calculator.description)
print(calculator.args)

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke({"input": "12와 23이 결혼하면?"})
#result = agent_executor.invoke({"input": "LINE JAPAN의 소유주는?"})

ConCat
두 수의 결혼 결과
{'a': {'title': 'A', 'description': 'first number', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second number', 'type': 'integer'}}


> Entering new AgentExecutor chain...

Invoking: `ConCat` with `{'a': 12, 'b': 23}`


1223


> Finished chain.


**사용자 정의 Tool 예시 #3**

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain import LLMChain

summarize_template = """아래의 글을 결론만 한 마디로 요약해 주세요.

{input}
"""
summarize_prompt = PromptTemplate(
   input_variables=["input"],
   template=summarize_template,
)

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)

tools = [
    Tool.from_function(
        func=summarize_chain.run,
        name="Summarizer",
        description="Text summarizer"
    ),
]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

text = """다음을 요약해 주세요.

안녕하세요! 저는 ChatGPT라고 불리는 AI 언어 모델입니다. OpenAI가 개발한 GPT-3.5 아키텍처를 기반으로 합니다. 저는 자연어 이해와 생성을 전문으로 하며, 다양한 주제에 대한 질문에 답하거나, 대화를 나누는 것을 잘합니다.
제 트레이닝 데이터는 2021년 9월까지의 정보를 기반으로 하기 때문에, 그 이후의 사건에 대해서는 지식이 없습니다. 하지만, 가능한 한 도움을 드리기 위해 노력할 것입니다.
질문이나 대화, 정보 공유 등, 어떤 도움이든 편하게 말씀해 주세요! 잘 부탁드립니다."""

result = agent_executor.invoke({"input": text})
print(result)

In [ ]:
# openai_functions_agent
from langchain.agents import load_tools
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)
tools = load_tools(["ddg-search"])
prompt = hub.pull("hwchase17/openai-functions-agent")

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({"input": "서울과 부산의 날씨를 알려줘"})
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': '서울 날씨'}`


Today's weather forecast for Seoul by the hour. 전국날씨. 기상청 바다예보 데이터가 제공됩니다. 해역별 마커에 표기되는 파고 정보는 현재 시점 기준입니다. 국내 날씨는 기상청, 해외 날씨는 아큐웨더・웨더뉴스・웨더채널 중 1개 제공사의 데이터가 제공됩니다. 국내 날씨는 기상청의 초단기실황, 초단기 ... 서울 (Seoul) ☀ 10일 기간 일기예보, 기상 정보, 웹카메라 화면, 일출 및 일몰, 지역의 풍속 및 강수량 Ventusky: 지도상 일기예보 Premium 날씨 문자 및 기상 조건의 예측: 조금 비 바람: 산들 바람 , 남쪽의 , 속도 3 초당 미터 돌풍: 11 초당 미터 상대 습도: 93% 흐림: 100% 기압: 1001 고전력 증폭기 강수량: 0,4 밀리미터 가시성: 18% 오늘 한낮에 서울 27도, 대전 32도 예상됩니다. 내일과 모레는 비가 잠시 소강상태에 들겠지만 곳곳으로 소나기가 내릴 전망입니다. ... 날씨 기사 ...
Invoking: `duckduckgo_search` with `{'query': '부산 날씨'}`


부산cc 날씨 컨트리클럽 주간일기예보를 알아보겠습니다. 현재 컨트리클럽 주변의 날씨 상황과 함께 실시간으로 반영되는 날씨 정보를 확인할 수 있으며, 주간 일별날씨예보와 비, 바람, 돌풍, 풍향, 눈 등의 세부적인 기상변화를 확인할 수 있습니다. Wednesday 31 July. 29° / 26°. 0.2 mm. 7 m/s. Open hourly forecast. Updated 10:45. Forecast as PDF Forecast as SVG. Weather forecast for Busan for the next 10 days. [날씨] 부산 내일도 온화해요…내일 아침까지 안개 주의 ... 뉴스7(부산) 온화한

In [ ]:
# openai_tools_agent
# agent의 동작 단계를 줄여서 속도 빠름 (LangSmith로 확인)
from langchain.agents import load_tools
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)
tools = load_tools(["ddg-search"])
prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_openai_tools_agent(llm, tools, prompt)  # <=
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({"input": "서울과 부산의 날씨를 알려줘"})
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'Seoul weather'}`


Get the current and forecast weather for Seoul, South Korea, including temperature, precipitation, wind, pressure, humidity, and more. See radar, satellite, and astronomy data for Seoul and nearby stations. Weather Today Weather Hourly 14 Day Forecast Yesterday/Past Weather Climate (Averages) Now. 70 °F. Clear. Feels Like: 70 °F Forecast: 74 / 64 °F Wind: 5 mph ↑ from West. Location: Seoul / Kimp'O International Airport. Current Time: Jun 7, 2024 at 9:51:33 pm. Seoul. Seoul Weather Forecast. Providing a local hourly Seoul weather forecast of rain, sun, wind, humidity and temperature. The Long-range 12 day forecast also includes detail for Seoul weather today. Live weather reports from Seoul weather stations and weather warnings that include risk of thunder, high UV index and forecast gales. Seoul features a Humid continental, severe dry winter, hot summer (Köppen classification: D